# Representing Text

We can represent text in many ways: character strings are a standard representation, but we can also create numerical representations of text. In this notebook we will discuss embeddings.

## Features

Features to any (machine learning) model can be continuous or categorical.

- We use continuous features to represent numerical values: income, number of times the user clicked on a link, prices, etc.
- Categorical features represent an instance of a class or category. They have a finite number of possible values: job title, genre of a movie, breed of a dog, etc.

## Embeddings

An **embedding** is a trained numerical representation of a categorical feature:

- We use the word *trained* to highlight that embeddings are learned during model training.
- Different models and training procedures can be used to obtained embeddings. Word2Vec and BERT embeddings, for example, are different and capture different characteristics of the features.

[OpenAI's documentation](https://platform.openai.com/docs/guides/embeddings) include a few uses of embeddings:


- Search: results are ranked by relevance to a query string.
- Clustering:  text strings are grouped by similarity.
- Recommendations:  items with related text strings are recommended.
- Anomaly detection:  outliers with little relatedness are identified.
- Diversity measurement: similarity distributions are analyze.
- Classification: text strings are classified by their most similar label.


### Tokenization

Embedding computations start with tokenization: representing the original text as tokens in a vocabulary. 

To illustrate the process, we can use the [`transformers`](https://huggingface.co/docs/transformers/en/index) library from [HuggingFace](https://huggingface.co/).

In [1]:
import transformers

documents = ["cats are fun"]

tokenizer = transformers.BertTokenizer.from_pretrained(
    'bert-base-uncased')
tokens = tokenizer(documents)
print(tokens)


{'input_ids': [[101, 8870, 2024, 4569, 102]], 'token_type_ids': [[0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


In the code snippet above, we used the `transformers` library to obtain the tokens that represent the phrase 'cats are fun'. The tokenizer returns a dictionary with an entry called `'input_ids'`, which contains an array of four integers. These integers are the positions of each token in the model's vocabulary. The vocabulary is [`'bert-base-uncased'`](https://huggingface.co/bert-base-uncased/blob/main/vocab.txt) and it is applied using the method `.from_pretrained()`.

We can show the vocabulary entries with:

In [2]:
print(f"Index of 'cats': {tokenizer.vocab['cats']}")
print(f"Index of 'are': {tokenizer.vocab['are']}")
print(f"Index of 'fun': {tokenizer.vocab['fun']}")

Index of 'cats': 8870
Index of 'are': 2024
Index of 'fun': 4569


IDs 101 and 102 are special tokens:

- ID 101 is the `[CLS]` token, indicating the begginning of a sequence.
- ID 102 is the `[SEP]` token, indicating the end of a sequence.

They are inserted automatically to the output of the BERT tokenizer. The BERT Tokenizer includes 30,522 unique tokens. In addition, the BERT tokenizer handles unkown tokens, `[UNK]`, using techniques such as WordPiece. You can read more about this tokenizer in [BERT Tokenization (Nowak, 2023)](https://tinkerd.net/blog/machine-learning/bert-tokenization/).

### Embeddings

The tokens obtained from the previous step are mapped to the model's precomputed embeddings. For each token in the model vocabulary, there is an embedding vector.

<div>
<img src="img/02_embeddings.png" width="700"/>
</div>


In [22]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
embedding_layer = model.embeddings
embedding_layer.word_embeddings.weight

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)

In [4]:
embedding_layer.word_embeddings.weight.shape

torch.Size([30522, 768])

The attribute `.weight` of the embedding layer shows the actual embeddings. It is a matrix of 30,522 rows and 768 columns (the object, in reality, is a 2-dimenional vector). 

+ The number of rows is equal to the size of the model vocabulary.
+ The number of columns is the hidden size or the size of the model's internal representation. 

In [23]:
tokens = tokenizer(['cats are fun'])
input_ids = tokens.input_ids[0]
input_ids

[101, 8870, 2024, 4569, 102]

In [27]:
doc_embeddings = embedding_layer.word_embeddings.weight[input_ids]
doc_embeddings

tensor([[ 0.0136, -0.0265, -0.0235,  ...,  0.0087,  0.0071,  0.0151],
        [-0.0590, -0.0339,  0.0108,  ..., -0.0328, -0.0285,  0.0624],
        [-0.0134, -0.0135,  0.0250,  ...,  0.0013, -0.0183,  0.0227],
        [-0.0073, -0.0459,  0.0314,  ..., -0.0196, -0.0372, -0.0150],
        [-0.0145, -0.0100,  0.0060,  ..., -0.0250,  0.0046, -0.0015]],
       grad_fn=<IndexBackward0>)

In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [ ]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")
else:
    client = OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
documents = [
    # Freedom
    "Freedom consists not in doing what we like, but in having the right to do what we ought.",
    "Those who deny freedom to others deserve it not for themselves.",
    "Liberty, when it begins to take root, is a plant of rapid growth.",
    "Freedom lies in being bold.",
    "Is freedom anything else than the right to live as we wish?",
    "I am no bird and no net ensnares me: I am a free human being with an independent will.",
    "The secret to happiness is freedom... And the secret to freedom is courage."
    "Freedom is the oxygen of the soul.", 
    "Life without liberty is like a body without spirit."
    # Friendship
    "There is nothing on this earth more to be prized than true friendship.",
    "There are no strangers here; Only friends you haven’t yet met.",
    "Friendship is the only cement that will ever hold the world together.",
    "A true friend is someone who is there for you when he'd rather be anywhere else.",
    "Friendship is the golden thread that ties the heart of all the world.", 
    "Your friend is the man who knows all about you and still likes you.",
    "A single rose can be my garden... a single friend, my world."
]

In [ ]:
embeddings = [get_embedding(doc) for doc in documents]

In [ ]:
embeddings

In [ ]:
embeddings_array = np.array(embeddings)
embeddings_array

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
reduced_embeddings = pca.fit_transform(embeddings_array)

In [ ]:
reduced_embeddings

In [ ]:
df = pd.DataFrame(reduced_embeddings, columns=["x", "y"]).assign(lables = documents)
ax = df.plot(kind='scatter', x='x', y='y', figsize=(8, 6))
for i, row in df.iterrows():
    ax.text(row['x'], row['y'], row['lables'], fontsize=9)